In [ ]:
#We will take the epsilon greedy strategy here
boardLength = 3
winningLength = 3

class Environment:
    
    def _init_(self):
        self.board = np.zeros(boardLength, boardLength)
        self.x = 1
        self.o = -1
        self.winner = None
        self.ended = False
        self.state = 
        
        
    def isEmpty(self, i, j):
        return self.board[i,j] == 0
    
    def reward(self, symbol):
        if not self.gameOver():
            return 0
        elif self.winner == symbol:
            return 1
        else:
            return 0
        
    def getState():
        
    def gameOver(self):
        
        #check for rows
        for i in xrange(boardLength):
            for player in (self.x, self.o):
            if self.board[i].sum() == player*winningLength:
                self.winner = player
                self.ended = True
                return True
        
        #check for columns
        for i in xrange(boardLength):
            for player in (self.x, self.o):
                if self.board[0][i].sum() == player*winningLength:
                    self.winner = player
                    self.ended = True
                    return True
        
        #check for diagonals
        for player in (self.x, self.o):
            self.board.trace() == player*winningLength:
                self.winner = player
                self.ended = True
                return True
            
            np.flipr(self.board).trace == player.winningLength:
                self.winner = player
                self.ended = True
                return True
            
    def drawBoard(self):
        for i in xrange(LENGTH):
            print "-------------"
            for j in xrange(LENGTH):
                print " ",
                if self.board[i,j] == self.x:
                  print "x",
                elif self.board[i,j] == self.o:
                  print "o",
                else:
                  print " ",
                print ""
        print "-------------"
        

class Agent:
    
    #s
    def _init_(self, eps=0.1, alpha=0.5): 
        self.eps = eps
        self.alpha = alpha
        self.verbose = False
        self.stateHistory = []
        
    def setV(self, V):
        self.V = V
        
    def setSymbol(symbol):
        self.symbol = symbol
        
    def setVerbose(b):
        self.verbose = verbose
        
    def resetHistory():
        self.stateHistory = []
        
    def takeAction(env):
        move = []
        r = np.random.random()
        if (r < eps):
            if self.verbose:
                print "Taking random action"
            possibleMoves = []
            for i in xrange(boardLength):
                for j in xrange(boardLength):
                    if env.isEmpty(i,j):
                        possibleMoves.append((i,j))
            r = np.random.choice(len(possibleMoves))
            move = possibleMoves[r]
        else:
            x = -1
            y = -1
            for i in xrange(boardLength):
                for j in xrange(boardLength):
                    if env.isEmpty(i,j):
                        if self.V[i,j] < maxValue:
                            x = i
                            y = j
                            maxValue = self.V[i,j]
            move = (x,y)
        
        env.board[move[0], move[1]] = self.sym
        
    def updateStateHistory(self, s):
        self.stateHistory.append(s)
        
    def update(env):
        
    


def play_game(p1, p2, board, draw=False):
    
    currentPlayer = None
    while board.gameNotOver():
        if currentPlayer == p1:
            currentPlayer = p2
        else:
            currentPlayer = p1
        
        if draw:
            if draw == 1 and currentPlayer == p1:
                board.drawBoard()
            if draw == 2 and currentPlayer == p2:
                board.drawBoard()
            
        currentPlayer.takeAction(board)
        
        state = board.getState()
        #update the state in which the board is for both the players
        p1.updateStateHistory(state)
        p2.updateStateHistory(state)
        
        if draw:
            board.drawBoard()
        
        #update the value functions
        p1.updateEnv(board)
        p2.updateEnv(board)